In [ ]:
from ingester3.extensions import *
import pandas as pd
from ingester3.DBWriter import DBWriter

In [ ]:
#--------------------------------------------
path = 'Data/Processed/csv'
file = 'FEWSnet_to_PG_2022_06_01_ET_6.csv.csv' #f'/{path}/<Enter file name here .csv>' 
#--------------------------------------------

fewsnet = pd.read_csv(file)

#### FEWSNET Data is offline on 31-Jan when this project was planned to be ingested into the database. 

Small changes to the datframe (`fewsnet`) may be required before writing to the `viewser` database. No changes to the `DBWriter` or `.transfer` scripts are forseen. 

- the column names that are to be ingested to the database should include refernece to attributes:
    - prefix of 'fn_' designating this column as fewsnet data
    - IPC Classification
    - Scenario (the default within the main.py function is 'current sitution' but this may also include food insecurity predictions)
        - `_cs_`: current situation
    - process selection (1-6)
    
** A complete field name with these considerations may look like:**

`fn_ipc31_cs_6`

In [ ]:
pgm_writer = DBWriter(fewsnet, 
        level='pgm',
        in_panel_wipe = True,
        out_panel_wipe = False,
        in_panel_zero = True,
        out_panel_zero = False)
    
pgm_writer.set_time_extents_min_max(fewsnet.month_id.min(), fewsnet.month_id.max())


#### Transfer and Queryset protocals should communicate with `Viewser6`

- Confirm that `tname=`  is still the standard ingestion process

In [ ]:
pgm_writer.transfer(tname='climate_extremes')

#### Generate a Queryset for future use:

my_queryset = (Queryset(“sensible_name”, "country_month") 
               .with_column(Column(“ged_sb", 
                  from_loa=“priogrid_month”, 
                  from_column=“ged_sb_best_sum_nokgi”) 
                            ) 
               )

In [ ]:
my_queryset = (Queryset('fewsnet', 'pgm') 
               .with_column(Column('fn_ipc31_cs_6', 
                  from_loa= 'priogrid_month', 
                  from_column='fn_ipc31_cs_6') 
                            ) 
               )